In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json  as js
from geopandas import GeoDataFrame
from shapely.geometry import Point
from shapely.ops import cascaded_union, unary_union
from shapely.geometry import Polygon, mapping
import datetime as dt
import glob
import csv
from dateutil import parser
import os
import itertools

In [146]:
b = pd.read_csv('report_f_all_v3.csv', sep = ';', encoding='utf8')
b2=b[b['buffer_out_ind']==1]
b2=b2[b2['60_min_ind']==1]
b2=b2[['time_key','ts','zid_from','zid_to','count_ctn']]

In [14]:
a = gpd.read_file('557_zone.SHP')
a['zid_from']=a['NO']
a['zid_to']=a['NO']

In [199]:
all_=b2[b2['time_key']=='2019-02-07']

In [216]:
all_['ts'].unique()

array(['2019-02-07 03:00:00', '2019-02-07 03:30:00',
       '2019-02-07 04:00:00', '2019-02-07 04:30:00',
       '2019-02-07 05:00:00', '2019-02-07 05:30:00',
       '2019-02-07 06:00:00', '2019-02-07 06:30:00',
       '2019-02-07 07:00:00', '2019-02-07 07:30:00',
       '2019-02-07 08:00:00', '2019-02-07 08:30:00',
       '2019-02-07 09:00:00', '2019-02-07 09:30:00',
       '2019-02-07 10:00:00', '2019-02-07 10:30:00',
       '2019-02-07 11:00:00', '2019-02-07 11:30:00',
       '2019-02-07 12:00:00', '2019-02-07 12:30:00',
       '2019-02-07 13:00:00', '2019-02-07 13:30:00',
       '2019-02-07 14:00:00', '2019-02-07 14:30:00',
       '2019-02-07 15:00:00', '2019-02-07 15:30:00',
       '2019-02-07 16:00:00', '2019-02-07 16:30:00',
       '2019-02-07 17:00:00', '2019-02-07 17:30:00',
       '2019-02-07 18:00:00', '2019-02-07 18:30:00',
       '2019-02-07 19:00:00', '2019-02-07 19:30:00',
       '2019-02-07 20:00:00', '2019-02-07 20:30:00',
       '2019-02-07 21:00:00', '2019-02-07 21:3

In [217]:
all_=all_[all_['ts']=='2019-02-07 19:30:00']

In [218]:
all_['ll']=all_['zid_from'].astype(str)+'_'+all_['zid_to'].astype(str)

In [219]:
len(all_['ll'].unique())

10679

In [258]:
#Перемещения между двумя ТР
def two_region:
    b3=b2[b2['time_key'] == '2019-04-11']
    from_156=b3[b3['zid_from']==150]
    to_156=b3[b3['zid_to']==150]
    to_156.head()
    from_156.to_csv('150.csv')
    from_156_morn=from_156[from_156['ts'].str.contains( ' 08:')]\
    .append(from_156[from_156['ts'].str.contains( ' 09:')])\
    .append(from_156[from_156['ts'].str.contains( ' 10:')])
    to_156_morn=to_156[to_156['ts'].str.contains( ' 08:')]\
    .append(to_156[to_156['ts'].str.contains( ' 09:')])\
    .append(to_156[to_156['ts'].str.contains( ' 10:')])\
    to_156_morn['ts'].unique()
    from_156_morn=pd.DataFrame(from_156_morn.groupby(['zid_to']).agg({'count_ctn': np.sum})).reset_index()
    to_156_morn=pd.DataFrame(to_156_morn.groupby(['zid_from']).agg({'count_ctn': np.sum})).reset_index()
    from_156_morn['count_ctn'].sum()
    from_156_morn=from_156_morn.merge(a, on='zid_to').drop(['NO'], axis=1)
    total1 = gpd.GeoDataFrame(from_156_morn, geometry='geometry')
    total1.crs='epsg:4326'
    total1.to_file('2019-10-17_from_150_morn_3hour.shp', driver='ESRI Shapefile')
    to_156_morn=to_156_morn.merge(a, on='zid_from').drop(['NO'], axis=1)
    total1 = gpd.GeoDataFrame(to_156_morn, geometry='geometry')
    total1.crs='epsg:4326'
    total1.to_file('2019-10-17_to_150_morn_3hour.shp', driver='ESRI Shapefile')

Генерим полигон без внутренних границ

In [ ]:
aaa = gpd.read_file('Samara2.shp', encoding="utf-8")
from shapely.ops import cascaded_union
polygons = aaa['geometry'].unique()
boundary = gpd.GeoSeries(cascaded_union(polygons))
from shapely.geometry import JOIN_STYLE
eps = 0.0005
boundary2 = boundary.buffer(eps, 1, join_style=JOIN_STYLE.mitre).buffer(-eps, 1, join_style=JOIN_STYLE.mitre)
boundary2.to_file('Sam.shp')
ss = gpd.GeoDataFrame(s2)
ss.crs='epsg:4326'
ss.to_file('Samara2.shp', driver='ESRI Shapefile', encoding='utf-8')

In [147]:
prop = pd.read_csv('prop2.csv',sep = ';', encoding='utf-8')

In [148]:
prop.head()

,Номер транспортного района,Муниципалитет (нижний уровень),Город/ Муниципалитет,Пригород чего,Км от 156 района
0,1,Красноглинский район,городской округ Самара,городской округ Самара,66.93
1,2,Советский район,городской округ Самара,городской округ Самара,6.16
2,3,Кировский район,городской округ Самара,городской округ Самара,9.05
3,4,городское поселение Смышляевка,Волжский район,пригород Самары,24.31
4,5,сельское поселение Подстёпки,Ставропольский район,пригород Тольятти,NaN


In [149]:
prop['zid_from']=prop['Номер транспортного района']
prop['zid_to']=prop['Номер транспортного района']
prop=prop.drop(['Номер транспортного района'],axis=1)

In [150]:
prop=prop.drop(['Муниципалитет (нижний уровень)'],axis=1)
prop=prop.drop(['Город/ Муниципалитет'],axis=1)

In [151]:
len(b2)

8237020

In [152]:
l=b2.merge(prop, on='zid_from')

In [153]:
len(l)

8237020

In [154]:
l=l.drop(['zid_to_y'],axis=1)

In [155]:
l['Пригород_FROM']=l['Пригород чего']
l['Км от 156 района_FROM']=l['Км от 156 района']
l=l.drop(['Пригород чего'],axis=1).drop(['Км от 156 района'],axis=1)
l['zid_to']=l['zid_to_x']
l=l.drop(['zid_to_x'],axis=1)

In [156]:
l.head()

,time_key,ts,zid_from,count_ctn,Пригород_FROM,Км от 156 района_FROM,zid_to
0,2019-02-07,2019-02-07 03:00:00,1,2,городской округ Самара,66.93,6
1,2019-02-07,2019-02-07 03:00:00,1,1,городской округ Самара,66.93,23
2,2019-02-07,2019-02-07 03:00:00,1,1,городской округ Самара,66.93,25
3,2019-02-07,2019-02-07 03:00:00,1,1,городской округ Самара,66.93,31
4,2019-02-07,2019-02-07 03:00:00,1,1,городской округ Самара,66.93,121


In [157]:
l2=l.merge(prop, on='zid_to')

In [158]:
l2.head()

,time_key,ts,zid_from_x,count_ctn,Пригород_FROM,Км от 156 района_FROM,zid_to,Пригород чего,Км от 156 района,zid_from_y
0,2019-02-07,2019-02-07 03:00:00,1,2,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6
1,2019-02-07,2019-02-07 04:00:00,1,1,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6
2,2019-02-07,2019-02-07 05:30:00,1,1,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6
3,2019-02-07,2019-02-07 05:30:00,1,1,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6
4,2019-02-07,2019-02-07 06:00:00,1,4,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6


In [159]:
l2['Пригород_TO']=l2['Пригород чего']

In [160]:
l2.head()

,time_key,ts,zid_from_x,count_ctn,Пригород_FROM,Км от 156 района_FROM,zid_to,Пригород чего,Км от 156 района,zid_from_y,Пригород_TO
0,2019-02-07,2019-02-07 03:00:00,1,2,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6,городской округ Жигулёвск
1,2019-02-07,2019-02-07 04:00:00,1,1,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6,городской округ Жигулёвск
2,2019-02-07,2019-02-07 05:30:00,1,1,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6,городской округ Жигулёвск
3,2019-02-07,2019-02-07 05:30:00,1,1,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6,городской округ Жигулёвск
4,2019-02-07,2019-02-07 06:00:00,1,4,городской округ Самара,66.93,6,городской округ Жигулёвск,NaN,6,городской округ Жигулёвск


In [161]:
l2=l2.drop(['Км от 156 района'],axis=1)

In [162]:
l2=l2.drop(['Пригород чего'],axis=1)

In [163]:
l2['zid_from']=l2['zid_from_x']

In [164]:
l2=l2.drop(['zid_from_x'],axis=1).drop(['zid_from_y'],axis=1)

In [165]:
l2.head()

,time_key,ts,count_ctn,Пригород_FROM,Км от 156 района_FROM,zid_to,Пригород_TO,zid_from
0,2019-02-07,2019-02-07 03:00:00,2,городской округ Самара,66.93,6,городской округ Жигулёвск,1
1,2019-02-07,2019-02-07 04:00:00,1,городской округ Самара,66.93,6,городской округ Жигулёвск,1
2,2019-02-07,2019-02-07 05:30:00,1,городской округ Самара,66.93,6,городской округ Жигулёвск,1
3,2019-02-07,2019-02-07 05:30:00,1,городской округ Самара,66.93,6,городской округ Жигулёвск,1
4,2019-02-07,2019-02-07 06:00:00,4,городской округ Самара,66.93,6,городской округ Жигулёвск,1


In [166]:
po_dnyam_all=pd.DataFrame(l2.groupby(['time_key']).agg({'count_ctn': np.sum})).reset_index().rename(columns={'count_ctn':'all'})

In [167]:
aglom=pd.read_csv('STA.csv', sep = ';', encoding='windows-1251')

In [168]:
sta=aglom['zid_from'].unique()

In [169]:
t_sta=l2[l2['zid_from'].isin(sta)]
t_sta=t_sta[t_sta['zid_to'].isin(sta)]

In [170]:
po_dnyam_sta=pd.DataFrame(t_sta.groupby(['time_key']).agg({'count_ctn': np.sum})).reset_index().rename(columns={'count_ctn':'sta'})

In [171]:
Samara_Samara=l2[(l2['Пригород_TO']=='городской округ Самара') & (l2['Пригород_FROM']=='городской округ Самара')]

In [172]:
po_dnyam_Samara=pd.DataFrame(Samara_Samara.groupby(['time_key']).agg({'count_ctn': np.sum})).reset_index().rename(columns={'count_ctn':'Samara'})

In [173]:
Tol_Tol=l2[(l2['Пригород_TO']=='городской округ Тольятти') & (l2['Пригород_FROM']=='городской округ Тольятти')]

In [174]:
po_dnyam_Tol=pd.DataFrame(Tol_Tol.groupby(['time_key']).agg({'count_ctn': np.sum})).reset_index().rename(columns={'count_ctn':'Tol'})

In [175]:
po_dnyam_all=po_dnyam_all.merge(po_dnyam_sta, on='time_key').merge(po_dnyam_Samara, on='time_key').merge(po_dnyam_Tol, on='time_key')

In [176]:
po_dnyam_all.head()

,time_key,all,sta,Samara,Tol
0,2019-02-07,1381152,1233143,457582,275995
1,2019-02-08,1646253,1467416,545406,333924
2,2019-02-09,1401156,1233279,448056,265784
3,2019-04-11,1717111,1537450,560408,342408
4,2019-04-12,1874988,1670334,596752,365128


In [177]:
po_dnyam_all.to_csv('po_dnyam.csv')

In [178]:
t_sta['count_ctn'].sum()

17996821

In [179]:
Samara_Tol=l2[(l2['Пригород_FROM']=='городской округ Самара') & (l2['Пригород_TO']=='городской округ Тольятти')]
Samara_Tol['count_ctn'].sum()

210338

In [180]:
Samara_Tol['time_key'].unique()

array(['2019-02-07', '2019-02-08', '2019-02-09', '2019-04-11',
       '2019-04-12', '2019-04-13', '2019-07-11', '2019-07-12',
       '2019-07-13', '2019-10-17', '2019-10-18', '2019-10-19'],
      dtype=object)

In [181]:
po_dnyam_Sam_Tol=pd.DataFrame(Samara_Tol.groupby(['time_key']).agg({'count_ctn': np.sum})).reset_index()

In [182]:
Tol_Samara=l2[(l2['Пригород_TO']=='городской округ Самара') & (l2['Пригород_FROM']=='городской округ Тольятти')]
Tol_Samara['count_ctn'].sum()

212672

In [183]:
po_dnyam_Tol_Sam=pd.DataFrame(Tol_Samara.groupby(['time_key']).agg({'count_ctn': np.sum})).reset_index()

In [184]:
po_dnyam_Sam_Tol=po_dnyam_Sam_Tol.merge(po_dnyam_Tol_Sam, on='time_key')

In [185]:
po_dnyam_Sam_Tol.to_csv('poezdki_Sam_Tol.csv')

In [186]:
Samara_prigorod=l2[(l2['Пригород_FROM']=='городской округ Самара') & (l2['Пригород_TO']=='пригород Самары')]
Samara_prigorod['count_ctn'].sum()

778451

In [187]:
prigorod_Samara=l2[(l2['Пригород_TO']=='городской округ Самара') & (l2['Пригород_FROM']=='пригород Самары')]
prigorod_Samara['count_ctn'].sum()

755826

In [188]:
Samara_Samara=l2[(l2['Пригород_TO']=='городской округ Самара') & (l2['Пригород_FROM']=='городской округ Самара')]
Samara_Samara['count_ctn'].sum()

6087029

In [189]:
Tol_Tol=l2[(l2['Пригород_TO']=='городской округ Тольятти') & (l2['Пригород_FROM']=='городской округ Тольятти')]
Tol_Tol['count_ctn'].sum()

3681276

In [190]:
Tol_prigorod=l2[(l2['Пригород_FROM']=='городской округ Тольятти') & (l2['Пригород_TO']=='пригород Тольятти')]
Tol_prigorod['count_ctn'].sum()

1057725

In [191]:
prigorod_Tol=l2[(l2['Пригород_TO']=='городской округ Тольятти') & (l2['Пригород_FROM']=='пригород Тольятти')]
prigorod_Tol['count_ctn'].sum()

1009217

In [192]:
for_clustrt = pd.read_csv('For_Cluster.csv',sep = ';', encoding='utf-8')

In [193]:
clustr = pd.read_csv('cluster.csv',sep = ';', encoding='utf-8')

In [194]:
clustr.head()

,До,Тип района
0,1,Пригород Тольятти
1,2,Самара
2,3,Самара
3,4,Пригород Самары
4,5,Пригород Тольятти


In [195]:
clustr['Тип района'].unique()

array(['Пригород Тольятти', 'Самара', 'Пригород Самары', 'Тольятти',
       'Периферия', 'Новокуйбышевск + Чапаевск'], dtype=object)

In [196]:
for_clustrt=for_clustrt.merge(clustr, on='До')

In [50]:
for_clustrt.head()

,От,До,id_2,TR_type,POP_FROM,POP_TO,WORK_TO,STUD_TO,RUSH,DAY,...,СТЗ_2,СТЗ_3,СТЗ_4,СТЗ_5,ТЦ_from,ТЦ_to,ТЦ-ТЦ,ТЦ_GR,Тип района_x,Тип района_y
0,1,3,1_3,3,8 131,20 729,17 411,954,10,54,...,442,399,397,439,0,4 708,4 708,"1,02",Пригород Тольятти,Самара
1,2,3,2_3,5,6 135,20 729,17 411,954,120,638,...,114,168,171,118,236,4 708,4 472,"1,02",Самара,Самара
2,4,3,4_3,5,1 030,20 729,17 411,954,14,53,...,299,369,373,304,0,4 708,4 708,"1,02",Пригород Самары,Самара
3,6,3,6_3,1,69 971,20 729,17 411,954,8,67,...,606,553,551,602,5 498,4 708,-790,1,Тольятти,Самара
4,16,3,16_3,1,101 142,20 729,17 411,954,7,86,...,693,690,690,693,131 427,4 708,-126 719,"0,52",Тольятти,Самара


In [55]:
clast1=for_clustrt[(for_clustrt['Тип района_x'].isin(['Пригород Тольятти','Тольятти'])) & (for_clustrt['Тип района_y'].isin(['Пригород Тольятти','Тольятти']))]

In [56]:
clast1.to_csv('clast1.csv')

In [59]:
clast2=for_clustrt[(for_clustrt['Тип района_x'].isin(['Пригород Самары','Самара'])) & (for_clustrt['Тип района_y'].isin(['Пригород Самары','Самара']))]
clast2.to_csv('clast2.csv')     

In [60]:
clast3=for_clustrt[~((for_clustrt['Тип района_x'].isin(['Пригород Самары','Самара'])) & (for_clustrt['Тип района_y'].isin(['Пригород Самары','Самара'])))]

In [61]:
clast3=clast3[~((for_clustrt['Тип района_x'].isin(['Пригород Тольятти','Тольятти'])) & (for_clustrt['Тип района_y'].isin(['Пригород Тольятти','Тольятти'])))]

C:\Users\mary_\Anaconda3\envs\Masha\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [71]:
clast5.to_csv('clast5.csv')   

In [67]:
clast4=clast3[~((for_clustrt['Тип района_x'].isin(['Пригород Тольятти','Тольятти','Пригород Самары','Самара'])) & (for_clustrt['Тип района_y'].isin(['Пригород Самары','Самара','Пригород Тольятти','Тольятти'])))]

C:\Users\mary_\Anaconda3\envs\Masha\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [70]:
clast5=clast3[((for_clustrt['Тип района_x'].isin(['Пригород Тольятти','Тольятти','Пригород Самары','Самара'])) & (for_clustrt['Тип района_y'].isin(['Пригород Самары','Самара','Пригород Тольятти','Тольятти'])))]

C:\Users\mary_\Anaconda3\envs\Masha\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [65]:
len(for_clustrt)-len(clast1)-len(clast2)

1008

In [ ]:
hour_Tol=pd.DataFrame(Tol_Tol_sut.groupby(['ts']).agg({'count_ctn': np.sum})).reset_index()
hour_Tol.to_csv('TOL.csv')
Samara=l2[l2['Пригород_FROM']=='городской округ Самара']
Samara_sut=Samara[Samara['time_key']=='2019-04-11']
hour_SAM=pd.DataFrame(Samara_sut.groupby(['ts']).agg({'count_ctn': np.sum})).reset_index()
hour_SAM.to_csv('Sam.csv')

Расчёт соседей по новым ID

In [5]:
ID=pd.read_csv('ID_NEW.csv',sep = ';', encoding='windows-1251')

In [6]:
ID

,zid_from,zid_to,ID
0,1,1,1
1,2,2,2
2,3,3,3
3,4,4,4
4,5,5,5
...,...,...,...
258,553,553,553
259,554,554,554
260,555,555,555
261,556,556,556


In [15]:
a=a.merge(ID, on ='zid_from')

In [16]:
a=a[['ID','geometry']]

In [17]:
a.head()

,ID,geometry
0,1,"POLYGON ((49.74624 53.47745, 49.74695 53.48435..."
1,2,"POLYGON ((50.26787 53.17875, 50.24173 53.20126..."
2,3,"POLYGON ((50.34918 53.23681, 50.35777 53.22709..."
3,4,"POLYGON ((50.34862 53.32653, 50.33540 53.32700..."
4,5,"POLYGON ((49.22210 53.59736, 49.21382 53.55950..."


In [18]:
a.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [19]:
a.to_file('POLIGONES_NEW.shp', driver='ESRI Shapefile', encoding='utf-8')

In [20]:
a_new = a.dissolve(by='ID')

In [22]:
a_new.to_file('POLIGONES_NEW2.shp', driver='ESRI Shapefile', encoding='utf-8')

In [23]:
a_new["NEIGHBORS"] = None  # add NEIGHBORS column

In [30]:
b = gpd.read_file('POLIGONES_NEW2.shp')

In [31]:
b.head()

,ID,geometry
0,1,"POLYGON ((49.74624 53.47745, 49.74695 53.48435..."
1,2,"POLYGON ((50.26787 53.17875, 50.24173 53.20126..."
2,3,"POLYGON ((50.34918 53.23681, 50.35777 53.22709..."
3,4,"POLYGON ((50.34862 53.32653, 50.33540 53.32700..."
4,5,"POLYGON ((49.22210 53.59736, 49.21382 53.55950..."


In [28]:
for index, row in a_new.iterrows():  
    neighbors = a_new[a_new.geometry.touches(row['geometry'])].ID.tolist() 
    neighbors = neighbors.remove(row.name)
    df.at[index, "my_neighbors"] = ", ".join(neighbors)

AttributeError: 'GeoDataFrame' object has no attribute 'ID'